# LDA Topic Modeling

In [1]:
# Standard Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# NLTK for Text Preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# Progress Bar
from tqdm import tqdm

# Gensim for LDA Topic Modeling
import gensim
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel

# PyLDAvis for Topic Visualization
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


The URLs below point to XLSX files containing tweets from 2022 from the following Twitter accounts:

- [@TheBeaverton](https://twitter.com/TheBeaverton)
- [@BillNye](https://twitter.com/BillNye) (Bill Nye)
- [@GeorgeTakei](https://twitter.com/GeorgeTakei) (George Takei)
- [@MeCookieMonster](https://twitter.com/MeCookieMonster) (Cookie Monster)
- [@neiltyson](https://twitter.com/neiltyson) (Neil deGrasse Tyson)
- [@StephenKing](https://twitter.com/StephenKing) (Stephen King)
- [@22_Minutes](https://twitter.com/22_Minutes) (This Hour Has 22 Minutes)

## Data Loading and Exploration

In [2]:
beaverton_url = 'https://drive.google.com/uc?export=download&id=173t6vMYcbSED81IOmg0MbhHsrSFSiF21'
billnye_url = 'https://drive.google.com/uc?export=download&id=1w1Xm-TQfcPfGICM_8_LQiWSqzf8WUk8N'
georgetakei_url = 'https://drive.google.com/uc?export=download&id=1_Ck9S2aW9cBQQrqoTiNLRmvRu4WvPn8y'
cookiemonster_url = 'https://drive.google.com/uc?export=download&id=10Av-HVklZA4Su3TcvSpxU6nS8At9mJ05'
neiltyson_url = 'https://drive.google.com/uc?export=download&id=1GSKxxldXe--HE4gSYAEn2ip2vS4bXKMM'
stephenking_url = 'https://drive.google.com/uc?export=download&id=1Oq2pUCv0d7x4tDKX1E-3hgdSAuXcZtVU'
twentytwo_minutes_url = 'https://drive.google.com/uc?export=download&id=1B2zwAJYdw_9skXAFbn7QIs6uyIikje5W'

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
beaverton = pd.read_excel(beaverton_url, engine='openpyxl')
beaverton.head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tweet-id                                         tweet-text   tweet-author  \
0         1  Heartwarming: This man reconnected with his mo...  The Beaverton   
1         2  OP-ED: Big Pharma put all of the chips in the ...  The Beaverton   
2         3  Make every day a 'talking to Americans about t...  The Beaverton   
3         4  Single drop of sweat falls down local dad’s fa...  The Beaverton   
4         5  100% of Canadians pretty sure they’ve had COVI...  The Beaverton   

  tweet-timestamp-date  
0           2022-03-05  
1           2022-03-04  
2           2022-03-04  
3           2022-03-04  
4           2022-03-03

In [4]:
billnye = pd.read_excel(billnye_url, engine='openpyxl')
billnye.head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tweet-id                                         tweet-text tweet-author  \
0         1  Happy Twosday, people(s). That’s right, it’s 2...     Bill Nye   
1         2  Dr. King would be 93 years old this week. The ...     Bill Nye   
2         3  Happy Isaac Newton’s Birthday James Webb Space...     Bill Nye   
3         4  Please, consider the following… my new VR Spac...     Bill Nye   
4         5  Winter tornadoes… warm winter in the south & c...     Bill Nye   

  tweet-timestamp-date  
0           2022-02-22  
1           2022-01-17  
2           2021-12-25  
3           2021-12-14  
4           2021-12-12

In [5]:
georgetakei = pd.read_excel(georgetakei_url, engine='openpyxl')
georgetakei.head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tweet-id                                         tweet-text tweet-author  \
0         1  He is still sorely missed. 💕 What is your favo...  georgetakei   
1         2  It’s terrifying but true. The GOP has become t...  georgetakei   
2         3                                This is not normal.  georgetakei   
3         4                             Very well put, indeed.  georgetakei   
4         5  With the former guy, mockery is the best polic...  georgetakei   

  tweet-timestamp-date  
0           2022-03-12  
1           2022-03-12  
2           2022-03-12  
3           2022-03-12  
4           2022-03-12

In [6]:
cookiemonster = pd.read_excel(cookiemonster_url, engine='openpyxl')
cookiemonster.head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tweet-id                                         tweet-text  \
0         1  Today, me so thankful for chefs like me friend...   
1         2  You can’t spell cookie without “oo.” Speaking ...   
2         3  Me so grateful @SesameErnie\n. Me think sharin...   
3         4  What do me love? Friends. Family. Cookies. Ple...   
4         5  Dipping cookies in milk? Me get it, but who ha...   

      tweet-author tweet-timestamp-date  
0  MeCookieMonster           2022-03-08  
1  MeCookieMonster           2022-02-25  
2  MeCookieMonster           2022-02-17  
3  MeCookieMonster           2022-02-14  
4  MeCookieMonster           2022-01-28

In [7]:
neiltyson = pd.read_excel(billnye_url, engine='openpyxl')
neiltyson.head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tweet-id                                         tweet-text tweet-author  \
0         1  Happy Twosday, people(s). That’s right, it’s 2...     Bill Nye   
1         2  Dr. King would be 93 years old this week. The ...     Bill Nye   
2         3  Happy Isaac Newton’s Birthday James Webb Space...     Bill Nye   
3         4  Please, consider the following… my new VR Spac...     Bill Nye   
4         5  Winter tornadoes… warm winter in the south & c...     Bill Nye   

  tweet-timestamp-date  
0           2022-02-22  
1           2022-01-17  
2           2021-12-25  
3           2021-12-14  
4           2021-12-12

In [8]:
stephenking = pd.read_excel(stephenking_url, engine='openpyxl')
stephenking.head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tweet-id                                         tweet-text tweet-author  \
0         1  What kind of dinosaur writes romance novels? A...  StephenKing   
1         2  Molly, aka the Thing of Evil, fools my friend ...  StephenKing   
2         3  Two questions (and a codicil) about MANIFEST: ...  StephenKing   
3         4  What most of us learned as kids on the playgro...  StephenKing   
4         5  I need baseball! Come on you guys, stop being ...  StephenKing   

  tweet-timestamp-date  
0           2022-03-05  
1           2022-02-25  
2           2022-02-25  
3           2022-02-23  
4           2022-02-16

In [9]:
twentytwo_minutes = pd.read_excel(twentytwo_minutes_url, engine='openpyxl')
twentytwo_minutes.head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tweet-id                                         tweet-text tweet-author  \
0         1  And in lighter news this week: Nope, sorry. We...    22Minutes   
1         2  Dee Snider of Twisted Sister says he approves ...    22Minutes   
2         3  Tim Hortons will be releasing their own line o...    22Minutes   
3         4  A new survey shows that 1 in 8 people suffer f...    22Minutes   
4         5  The Bank of Canada has raised its interest rat...    22Minutes   

  tweet-timestamp-date  
0  2022-03-03 00:00:00  
1  2022-03-03 00:00:00  
2  2022-03-03 00:00:00  
3  2022-03-02 00:00:00  
4  2022-03-02 00:00:00

In [10]:
beaverton['account'] = 'The Beaverton'
billnye['account'] = 'Bill Nye'
georgetakei['account'] = 'George Takei'
cookiemonster['account'] = 'Cookie Monster'
neiltyson['account'] = 'Neil deGrasse Tyson'
stephenking['account'] = 'Stephen King'
twentytwo_minutes['account'] = 'This Hour Has 22 Minutes'

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
data = pd.concat([beaverton, billnye, georgetakei, cookiemonster, neiltyson, stephenking, twentytwo_minutes], ignore_index=True)
data.head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tweet-id                                         tweet-text   tweet-author  \
0         1  Heartwarming: This man reconnected with his mo...  The Beaverton   
1         2  OP-ED: Big Pharma put all of the chips in the ...  The Beaverton   
2         3  Make every day a 'talking to Americans about t...  The Beaverton   
3         4  Single drop of sweat falls down local dad’s fa...  The Beaverton   
4         5  100% of Canadians pretty sure they’ve had COVI...  The Beaverton   

  tweet-timestamp-date        account  
0  2022-03-05 00:00:00  The Beaverton  
1  2022-03-04 00:00:00  The Beaverton  
2  2022-03-04 00:00:00  The Beaverton  
3  2022-03-04 00:00:00  The Beaverton  
4  2022-03-03 00:00:00  The Beaverton

In [12]:
data.tail()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tweet-id                                         tweet-text  \
1071       196  A student at U of T is launching a city-wide s...   
1072       197  New poll shows nearly half of Canadians don’t ...   
1073       198  Apple has introduced the iPhone 13! Huge! Mass...   
1074       199  The Price Is Right is celebrating its 50th yea...   
1075       200  Some restaurants are reporting a shortage of c...   

     tweet-author tweet-timestamp-date                   account  
1071    22Minutes  2021-09-17 00:00:00  This Hour Has 22 Minutes  
1072    22Minutes  2021-09-17 00:00:00  This Hour Has 22 Minutes  
1073    22Minutes  2021-09-16 00:00:00  This Hour Has 22 Minutes  
1074    22Minutes  2021-09-15 00:00:00  This Hour Has 22 Minutes  
1075    22Minutes  2021-09-15 00:00:00  This Hour Has 22 Minutes

In [13]:
data.info()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076 entries, 0 to 1075
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   tweet-id              1076 non-null   int64 
 1   tweet-text            1076 non-null   object
 2   tweet-author          1076 non-null   object
 3   tweet-timestamp-date  1076 non-null   object
 4   account               1076 non-null   object
dtypes: int64(1), object(4)
memory usage: 42.2+ KB


## Text Preprocessing

In [14]:
# to initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text) # [^a-zA-Z\s] => This pattern matches any character that is not an alphabetic letter (A to Z or a to z) or a whitespace.

    tokens = word_tokenize(text) # A function from NLTK that splits a string of text into individual words or tokens (smallest units of meaning).

    tokens = [token.lower() for token in tokens]

    tokens = [token for token in tokens if token not in stop_words]

    tokens = [lemmatizer.lemmatize(token) for token in tokens] # Converts each word (token) into its root form (lemma).

    return tokens

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
tqdm.pandas() # tqdm.pandas() integrates the tqdm progress bar with pandas, allowing you to track progress when using .apply() on DataFrames or Series.
data['processed_tweet_text'] = data['tweet-text'].progress_apply(preprocess_text)
data[['tweet-text', 'processed_tweet_text']].head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 1076/1076 [00:08<00:00, 122.82it/s]


tweet-text  \
0  Heartwarming: This man reconnected with his mo...   
1  OP-ED: Big Pharma put all of the chips in the ...   
2  Make every day a 'talking to Americans about t...   
3  Single drop of sweat falls down local dad’s fa...   
4  100% of Canadians pretty sure they’ve had COVI...   

                                processed_tweet_text  
0  [heartwarming, man, reconnected, mom, could, a...  
1  [oped, big, pharma, put, chip, vaccine, cant, ...  
2  [make, every, day, talking, american, healthca...  
3  [single, drop, sweat, fall, local, dad, face, ...  
4   [canadian, pretty, sure, theyve, covid, already]

##  Corpus Preparation, LDA Model Training, and Topic Evaluation

This section prepares the processed text for LDA topic modelling by creating a structured corpus and training the model using Gensim.

In [17]:
# Creates a dictionary where each unique word gets a unique integer ID.
# The dictionary stores how many times each word appears across all documents.
id2word = corpora.Dictionary(data['processed_tweet_text'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Converts each document (text, which is a list of words) into a sparse vector.
corpus = [id2word.doc2bow(text) for text in data['processed_tweet_text']]
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
[[(id2word[i], freq) for i, freq in doc] for doc in corpus[:1]]

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[('ask', 1),
  ('could', 1),
  ('heartwarming', 1),
  ('man', 1),
  ('mom', 1),
  ('reconnected', 1),
  ('tax', 1)]]

### Why These Parameters?

- `num_topics=5`: Each account likely discusses a few main themes. Using five topics per account helps keep topics clear and meaningful without making them too broad or too specific. **Experimenting with coherence scores** can determine the optimal number.
- `passes=50`: Runs the model enough times to find good topics without wasting time on extra passes that don't add much improvement.
- `update_every=1`: Updates the model after each batch of documents, making training faster and more memory-efficient, especially for large datasets.
- `chunksize=200`: Processes 200 documents at a time, which makes training faster and helps the model find more stable topics.
- `alpha='auto'`: Automatically adjusts how many topics each document contains, so the model can decide if a document should focus on just one topic or multiple topics.
- `per_word_topics=False`: Speeds up processing by skipping extra details about individual words, which are not usually needed for topic analysis.

In [20]:
# Train a single LDA model on all accounts together
lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=15,  # Adjust based on coherence score testing
    passes=50,
    update_every=1,
    chunksize=200,
    alpha='auto',
    per_word_topics=False
)

# The coherence score measures how well the words in a topic are related, helping us evaluate the quality of the LDA model.
# A higher coherence score means better topics, making the model more useful!
# The coherence score in LDA topic modeling typically ranges between 0 and 1, but it does not have strict theoretical bounds.
# A higher coherence score (~0.5 to 0.7) means the topics are well-formed and useful!
coherence_model = CoherenceModel(
    model=lda_model,
    texts=data['processed_tweet_text'],
    dictionary=id2word,
    coherence='c_v'
)
coherence_score = coherence_model.get_coherence()

# Print the overall coherence score and topics
print(f"LDA Model Trained on All Accounts (Coherence Score: {coherence_score:.4f})")
print("=" * 100)

for topic_id, topic in lda_model.show_topics(formatted=True, num_words=10):
    print(f"Topic {topic_id + 1}: {topic}")

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LDA Model Trained on All Accounts (Coherence Score: 0.4754)
Topic 15: 0.054*"much" + 0.052*"game" + 0.041*"cant" + 0.037*"never" + 0.030*"im" + 0.023*"ago" + 0.021*"gas" + 0.021*"anyone" + 0.017*"bad" + 0.016*"win"
Topic 1: 0.038*"space" + 0.037*"national" + 0.029*"nasa" + 0.027*"birthday" + 0.024*"human" + 0.019*"exploreplanets" + 0.019*"ready" + 0.018*"exploration" + 0.018*"best" + 0.015*"changed"
Topic 2: 0.051*"change" + 0.037*"climate" + 0.033*"could" + 0.031*"little" + 0.027*"say" + 0.025*"want" + 0.024*"world" + 0.017*"found" + 0.017*"house" + 0.015*"probably"
Topic 11: 0.037*"lot" + 0.031*"think" + 0.031*"best" + 0.030*"youre" + 0.030*"know" + 0.027*"still" + 0.025*"last" + 0.020*"sound" + 0.019*"year" + 0.017*"story"
Topic 6: 0.045*"got" + 0.024*"news" + 0.023*"toronto" + 0.023*"oh" + 0.022*"new" + 0.018*"whole" + 0.015*"least" + 0.015*"average" + 0.014*"boy" + 0.014*"away"
Topic 3: 0.025*"two" + 0.024*"love" + 0.019*"ontario" + 0.019*"stop" + 0.016*"big" + 0.016*"there" + 0.0

In [21]:
# Function to compute coherence values for different topic numbers
def compute_coherence_values(dictionary, corpus, texts, start=2, limit=40, step=4):
    coherence_values = []
    model_list = []
    num_topics_list = range(start, limit + 1, step)

    for num_topics in num_topics_list:
        model = gensim.models.LdaModel(
            corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=50,
            update_every=1, chunksize=200, alpha='auto', per_word_topics=False)
        model_list.append(model)

        coherence_model = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherence_model.get_coherence())

    return model_list, coherence_values, num_topics_list

# Run the function to compute coherence scores for various topic numbers
model_list, coherence_values, num_topics_list = compute_coherence_values(
    dictionary=id2word,
    corpus=corpus,
    texts=data['processed_tweet_text'],
    start=2,
    limit=40,
    step=4
)

# Plot coherence scores
plt.figure(figsize=(10, 6))
plt.plot(num_topics_list, coherence_values, marker='o', linestyle='-')
plt.xlabel("Number of Topics")
plt.ylabel("Coherence Score")
plt.title("Optimal Number of Topics Based on Coherence Score")
plt.legend(["Coherence Score"], loc='best')
plt.grid(True);

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


It looks like the **coherence score increases with the increase in the number of topics**. We will use the model with the highest coherence score:

In [22]:
# Find the index of the best coherence score
best_result_index = coherence_values.index(max(coherence_values))

# Select the optimal model based on the highest coherence score
optimal_model = model_list[best_result_index]

# Get the best number of topics
best_num_topics = num_topics_list[best_result_index]

# Print the best number of topics and its coherence score
print(f"The optimal number of topics is {best_num_topics}, achieving the highest coherence score of {coherence_values[best_result_index]:.4f}.")

# Display the top topics from the optimal model
print("=" * 100)
for topic_id, topic in optimal_model.show_topics(formatted=True, num_words=10):
    print(f"Topic {topic_id + 1}: {topic}")

The optimal number of topics is 18, achieving the highest coherence score of 0.4836.
Topic 13: 0.050*"guy" + 0.038*"way" + 0.024*"true" + 0.019*"real" + 0.017*"claim" + 0.017*"photo" + 0.012*"head" + 0.012*"faster" + 0.010*"crime" + 0.009*"halloween"
Topic 11: 0.055*"hour" + 0.026*"tonight" + 0.023*"thinking" + 0.023*"dad" + 0.021*"use" + 0.021*"clown" + 0.020*"challenge" + 0.018*"problem" + 0.014*"offer" + 0.013*"hot"
Topic 9: 0.121*"canadian" + 0.039*"set" + 0.037*"celebrate" + 0.025*"bad" + 0.024*"hand" + 0.023*"every" + 0.019*"best" + 0.016*"step" + 0.015*"piece" + 0.011*"year"
Topic 4: 0.044*"want" + 0.039*"even" + 0.034*"food" + 0.028*"price" + 0.028*"remember" + 0.028*"vote" + 0.021*"water" + 0.019*"clean" + 0.017*"wonder" + 0.016*"win"
Topic 18: 0.085*"canada" + 0.069*"friend" + 0.047*"love" + 0.033*"ever" + 0.025*"without" + 0.024*"mean" + 0.019*"close" + 0.017*"favourite" + 0.017*"vaccine" + 0.017*"parent"
Topic 15: 0.098*"one" + 0.036*"new" + 0.031*"know" + 0.031*"sure" + 0.

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Let's see how the words are clustered using pyLDAVis.

In [23]:
# Creating Topic Distance Visualization
p = gensimvis.prepare(optimal_model, corpus, id2word)
p

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.148321  0.294397       1        1  11.528658
6      0.181460 -0.163386       2        1   9.711785
16     0.132492 -0.098673       3        1   9.614389
5      0.033730  0.020498       4        1   8.251410
4      0.139068 -0.052058       5        1   7.423155
11     0.038839 -0.003310       6        1   7.322320
14    -0.030568 -0.014011       7        1   7.094025
1      0.021534  0.066199       8        1   6.008975
2     -0.062712 -0.029234       9        1   4.866681
7      0.026142 -0.000019      10        1   4.250093
0     -0.064770 -0.048863      11        1   4.031776
9     -0.060877  0.075355      12        1   3.777863
13    -0.108608 -0.011188      13        1   3.604719
3     -0.066465 -0.025714      14        1   3.112209
17    -0.099223  0.002458      15        1   3.003235
10    -0.064381 -0.013124      16        1   2.394602
8     -0.082360  0.002083      17        1   2.217365
12    -0.081622 -0.001409      18        1   1.786740, topic_info=           Term        Freq       Total Category  logprob  loglift
17          day  160.000000  160.000000  Default  30.0000  30.0000
150         say  117.000000  117.000000  Default  29.0000  29.0000
380        like  111.000000  111.000000  Default  28.0000  28.0000
668         one  101.000000  101.000000  Default  27.0000  27.0000
92         time   88.000000   88.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
607         way    8.675427   26.338201  Topic18  -3.2747   2.9143
599  commercial    1.569031    4.680390  Topic18  -4.9847   2.9319
711       happy    1.508958   65.037812  Topic18  -5.0237   0.2612
691          im    1.246016   23.504346  Topic18  -5.2152   1.0875
826     science    1.188789   48.200067  Topic18  -5.2622   0.3224

[749 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
839       8  0.927305          able
925      14  0.867092        access
477      11  0.763418  accidentally
815      16  0.703243    accomplish
926      14  0.592788       account
...     ...       ...           ...
386       1  0.187495         youre
386       7  0.562484         youre
386       9  0.062498         youre
386      10  0.156246         youre
624      15  0.858866          zoom

[905 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 7, 17, 6, 5, 12, 15, 2, 3, 8, 1, 10, 14, 4, 18, 11, 9, 13])